### Step 1: Import libraries and load X and y


In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import pandas as pd
import joblib

# Load dataset
df = pd.read_csv('../data/medical_reports.csv')

# Extract labels
labels = df.columns[1:]
y = df[labels]

# TF-IDF vectorization
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['report_text'].str.lower().str.replace('\n', ' ', regex=True))

# Print to confirm
print("TF-IDF Shape:", X.shape)
print("Labels:", labels.tolist())


TF-IDF Shape: (200, 42)
Labels: ['diabetes', 'hypertension', 'pneumonia', 'asthma', 'arthritis']


### Step 2: Split data into train and test sets


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape


((160, 42), (40, 42))

### Step 3: Train a multilabel Logistic Regression model


In [6]:
model = OneVsRestClassifier(LogisticRegression())
model.fit(X_train, y_train)


OneVsRestClassifier(estimator=LogisticRegression())

### Step 4: Predict and evaluate the model performance


In [7]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=labels))


              precision    recall  f1-score   support

    diabetes       0.46      0.67      0.55        18
hypertension       0.61      0.76      0.68        25
   pneumonia       0.46      0.55      0.50        20
      asthma       0.51      0.90      0.65        20
   arthritis       0.46      0.68      0.55        19

   micro avg       0.51      0.72      0.59       102
   macro avg       0.50      0.71      0.59       102
weighted avg       0.51      0.72      0.59       102
 samples avg       0.50      0.69      0.55       102



C:\Users\User\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
